<a href="https://colab.research.google.com/github/amifunny/Deep-Learning-Notebook/blob/master/Image_GAN_HD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# kaggle requirements
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

"""
  ** This Notebook uses Kaggle Dataset. 
  If not a user , Sign in here - 'https://www.kaggle.com/'
  If already a user , Obtain your api_token by - 'https://www.kaggle.com/docs/api#authentication
""" 
api_token = {"username":"<your_user_name>","key":"<key_here>"}

import json

with open("/root/.kaggle/kaggle.json",'w') as file:
    json.dump( api_token , file )
    file.close()

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
import tensorflow as tf
import numpy as np

import kaggle

from PIL import Image
import os
import matplotlib.pyplot as plt
from IPython import display

In [0]:
# dataset at 'https://www.kaggle.com/c/dogs-vs-cats/data
# !kaggle competitions download -c dogs-vs-cats

!kaggle datasets download -d abhikjha/utk-face-cropped

 97% 225M/232M [00:04<00:00, 48.5MB/s]
100% 232M/232M [00:04<00:00, 50.9MB/s]


In [0]:
# **!unzip train.zip -d data

!unzip utk-face-cropped.zip -d face_data


In [0]:
# takes arg "test1" or "train", and noi(number of images)
# def get_data( data_type, start=0,end=1000):

#     master_dir = '/content/data/'+ data_type +'/'

#     all_images = os.listdir(master_dir)[start:end]

#     data_x = []

#     for img_file in all_images:

#         img = Image.open(master_dir+img_file)
#         img_arr = ( np.array( img.resize((96,96)) )/255.0 )
#         data_x.append( img_arr )

#     return data_x

def get_data( data_type, start=0,end=1000):

    master_dir = '/content/face_data/'+ data_type +'/'

    all_images = os.listdir(master_dir)[start:end]

    data_x = []

    for img_file in all_images:

        try:
          img = Image.open(master_dir+img_file)
          img_arr = ( np.array( img.resize((96,96)) )/255.0 )
          data_x.append( img_arr )
        except:
          continue

    return data_x

In [0]:
# domain_imgs = get_data( "train",0,-1 )
domain_imgs = get_data( 'utkcropped' , 0,10000 )

In [0]:
len(domain_imgs)

In [0]:
# check the data
show = plt.imshow( domain_imgs[1] )
plt.show()

In [0]:
def prep_batch(dom_imgs,batch_size):

    num_of_batches = int(len(dom_imgs)/batch_size)
    
    img_shape = dom_imgs[0].shape
    dom_batches = []
    
    for i in range(num_of_batches):
        
        dom_batch = tf.convert_to_tensor( dom_imgs[i*batch_size:(i+1)*batch_size] )
        dom_batches.append( dom_batch )

    return dom_batches,num_of_batches

In [0]:
"""
  DISCRIMINATOR MODEL
"""

inputs = tf.keras.layers.Input((96,96,3))

out = tf.keras.layers.Conv2D( 32,(5,5),strides=(2,2) , padding='same' )(inputs)
out = tf.keras.layers.LeakyReLU()(out)
out = tf.keras.layers.Dropout(0.2)(out)


out = tf.keras.layers.Conv2D( 64,(5,5),strides=(2,2), padding='same' )(out)
out = tf.keras.layers.LeakyReLU()(out)
out = tf.keras.layers.Dropout(0.2)(out)


out = tf.keras.layers.Conv2D( 128,(5,5),strides=(2,2), padding='same' )(out)
out = tf.keras.layers.LeakyReLU()(out)
out = tf.keras.layers.Dropout(0.2)(out)

out = tf.keras.layers.Conv2D( 256,(5,5),strides=(2,2), padding='same' )(out)
out = tf.keras.layers.LeakyReLU()(out)
out = tf.keras.layers.Dropout(0.2)(out)

out = tf.keras.layers.Flatten()(out)
outputs = tf.keras.layers.Dense( 1 )(out)

D_model=0
D_model = tf.keras.Model(inputs=inputs,outputs=outputs)
D_model.summary()

In [0]:
"""
  GENERATOR MODEL
"""


inputs = tf.keras.layers.Input((100,))

out = tf.keras.layers.Dense(3*3*1024)(inputs)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

out = tf.keras.layers.Reshape(target_shape=(3,3,1024))(out)

out = tf.keras.layers.Conv2DTranspose(256,(5,5),strides=(2,2),padding="same",use_bias=False)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

out = tf.keras.layers.Conv2DTranspose(128,(5,5),strides=(2,2),padding="same",use_bias=False)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

out = tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding="same",use_bias=False)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

out = tf.keras.layers.Conv2DTranspose(32,(5,5),strides=(2,2),padding="same",use_bias=False)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

outputs = tf.keras.layers.Conv2DTranspose(3,(5,5),strides=(1,1),use_bias=False,activation='tanh',padding="same")(out)

G_model = 0
G_model=tf.keras.Model(inputs=inputs,outputs=outputs)
G_model.summary()

In [0]:
try_gen = G_model( tf.random.normal([1,100]) , training=False )
print(try_gen.shape)
# print( try_gen )
# print( tf.cast( try_gen[0,:,:,:]*127.5+127.5 , tf.uint8 ) )
show = plt.imshow( tf.cast( try_gen[0,:,:,:]*127.5+127.5 , tf.uint8 ) )
plt.show()

In [0]:
seed = tf.random.normal([ 16 , 100 ])

In [0]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow( tf.cast( predictions[i, :, :, :] * 127.5 + 127.5 , tf.uint8 )  , cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [0]:
#********************
batch_size = 128
l_rate = 0.0001
ctr=0
epochs=50
# k -factor
k=1
subset_size = 1024
#********************

d_optimizer = tf.keras.optimizers.Adam( l_rate )
g_optimizer = tf.keras.optimizers.Adam( l_rate )

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


with tf.device('/device:GPU:0'):

  for e in range(epochs):
    
    print("********* EPOCH :: {}".format(e))

    # display.clear_output(wait=True)
    generate_and_save_images(G_model,
                             e + 1,
                             seed)

    g_cost = 0
    start = 0
    end = start + subset_size

    while True:

      subset = domain_imgs[start:end]
      if end>10000:
        break
      start = end
      end = start + subset_size
      batches,_ = prep_batch(subset,batch_size)

      for batch_true in batches:
     
        batch_noise = tf.random.normal( (batch_size,100 ) )

        with tf.GradientTape(persistent=True) as tape:

          pred_true = tf.squeeze( D_model( batch_true , training=True ) )
          batch_false = G_model(batch_noise, training=True)
          pred_false = tf.squeeze( D_model(batch_false, training=True) )

          d_cost = discriminator_loss(pred_true,pred_false)
          g_cost = generator_loss(pred_false)

        if ctr%10==0:

          print("optimally 1 for D : {}".format(pred_true[:10]))
          print("optimally 0 for D : {}".format(pred_false[:10]))
          print("D_Loss is === >  {}".format(d_cost))
          print("G_Loss is === >  {}".format(g_cost))

          generate_and_save_images(G_model,
                             e + 1,
                             seed)

        ctr = ctr+1
        
        if ctr%k==0:
          d_grads = tape.gradient( d_cost,D_model.trainable_variables )
          d_optimizer.apply_gradients(zip(d_grads,D_model.trainable_variables))

        g_grads = tape.gradient( g_cost,G_model.trainable_variables )
        g_optimizer.apply_gradients(zip(g_grads,G_model.trainable_variables))


In [0]:
"""
  Train it for 100 Epochs & see the results
  Somewhat Good Right! If you are not getting good results , Restart the Notebook and Make Sure you don't Change anything.
  GAN's are sensitive and give results only when every aspect is right eg. Activations,Dropout,Loss
  See -- 'https://github.com/soumith/ganhacks'

  Feel Free to Try Another Dataset : )

"""